In [1]:
! python --version

Python 3.8.12


In [2]:
!pip install pyspark

  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=5de340d1c65dd7fae9f64909efb1d44d29898208028ad24cc84d470918a873fc
  Stored in directory: c:\users\zieglejn\appdata\local\pip\cache\wheels\58\94\83\915c9059e4b038e2d43a6058f307fe1c3e8536e5745f3b23b7
Successfully built pyspark


In [3]:
import pyspark

In [5]:
! PWD

Der Befehl "PWD" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.
